# Feature Engineering
Generate rolling-window features and save the feature table.

In [7]:
import sys
from pathlib import Path

sys.path.append(str(Path("..").resolve()))

from src.feature_engineering import build_features, assign_seismic_zone
import pandas as pd

candidates = [
    Path("../data/processed/usgs_india_clean.csv"),
    Path("data/processed/usgs_india_clean.csv"),
    Path("/workspaces/Earthquake-alert-system/earthquake_model/data/processed/usgs_india_clean.csv"),
]
data_path = next((p for p in candidates if p.exists()), None)
if data_path:
    df = pd.read_csv(data_path)
    df["time"] = pd.to_datetime(df["time"], utc=True, errors="coerce")
    df = df.dropna(subset=["time"]).reset_index(drop=True)
    features = build_features(df)
    output_path = Path("../data/processed/features.csv")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    features.to_csv(output_path, index=False)
    features.head()
else:
    print("Cleaned data not found yet. Run 01_data_fetch.ipynb first if you want features.")

In [6]:
# Zone Verification — Run this before training
test_points = [
    {"city": "Bhuj, Gujarat",      "lat": 23.2420, "lon": 69.6669, "expected": 5},
    {"city": "Delhi",              "lat": 28.6139, "lon": 77.2090, "expected": 4},
    {"city": "Chennai",            "lat": 13.0827, "lon": 80.2707, "expected": 3},
    {"city": "Guwahati, Assam",    "lat": 26.1445, "lon": 91.7362, "expected": 5},
    {"city": "Mumbai",             "lat": 19.0760, "lon": 72.8777, "expected": 3},
    {"city": "Kolkata",            "lat": 22.5726, "lon": 88.3639, "expected": 3},
    {"city": "Jaisalmer",          "lat": 26.9157, "lon": 70.9083, "expected": 2},
]

print(f"{'City':<25} {'Expected':<10} {'Got':<10} {'Status'}")
print("-" * 55)
all_pass = True
for p in test_points:
    got = assign_seismic_zone(p["lat"], p["lon"])
    status = "PASS" if got == p["expected"] else "FAIL"
    if got != p["expected"]:
        all_pass = False
    print(f"{p['city']:<25} Zone {p['expected']:<7} Zone {got:<7} {status}")

print()
print("All zones verified — safe to proceed with training!" if all_pass
      else "Fix zone boundaries before training — do NOT proceed yet!")

City                      Expected   Got        Status
-------------------------------------------------------
Bhuj, Gujarat             Zone 5       Zone 5       PASS
Delhi                     Zone 4       Zone 4       PASS
Chennai                   Zone 3       Zone 3       PASS
Guwahati, Assam           Zone 5       Zone 5       PASS
Mumbai                    Zone 3       Zone 3       PASS
Kolkata                   Zone 3       Zone 3       PASS
Jaisalmer                 Zone 2       Zone 2       PASS

All zones verified — safe to proceed with training!
